In [1]:
!pip install selenium gspread oauth2client

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.1/9.7 MB 1.3 MB/s eta 0:00:08
   - -------------------------------------- 0.3/9.7 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/9.7 MB 2.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/9.7 MB 3.1 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/9.7 MB 3.4 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.7 MB 5.0 MB/s eta 0:00:02
   ---------- ----------------------------- 2.5/9.7 MB 7.0 MB/s eta 0:00:02
   ------------------- -------------------- 4.8/9.7 MB 11.9 MB/s eta 0:00:01
   ----------------------------------- ---- 8.5/9.7 MB 18.7 MB/s eta 0:00:01
   ----------------------------------- ---- 8.5/9.7 MB 18.7 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 17.7 MB/s eta 0:00:00
   ------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\braag\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
!pip install webdriver-manager


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\braag\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import urllib.parse
import time
from webdriver_manager.chrome import ChromeDriverManager


# Step 1: Google Sheets Authorization
def authorize_google_sheets(credentials_file, sheet_url):
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive",
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(creds)
    sheet = client.open_by_url(sheet_url).sheet1
    return sheet

# Step 2: Clean Phone Numbers
def clean_phone_number(phone):
    phone = phone.strip()
    phone = ''.join(c for c in phone if c.isdigit() or c == '+')
    return phone

# Step 3: Send WhatsApp Messages Using Selenium
def send_whatsapp_messages(sheet, driver, delay=15):
    # Fetch all data from the sheet
    data = sheet.get_all_records()
    print("Fetched Data from Google Sheet:", data)

    driver.get("https://web.whatsapp.com")
    input("Scan QR code on WhatsApp Web and press Enter here...")

    for row in data:
        try:
            name = row["Name"]
            raw_phone = row["Number"]
            phone = clean_phone_number(raw_phone)  # Clean the phone number

            # Validate phone number format
            if not phone.startswith("+") or not phone[1:].isdigit() or len(phone) < 10:
                print(f"Invalid phone number for {name}: {raw_phone}")
                continue

            # Generate WhatsApp link with pre-filled message
            message = f"Hello {name}, this is an automated message from us."
            encoded_message = urllib.parse.quote(message)
            whatsapp_link = f"https://web.whatsapp.com/send?phone={phone}&text={encoded_message}"

            # Open WhatsApp chat
            driver.get(whatsapp_link)
            time.sleep(10)  # Wait for the chat to load

            # Click the send button
            send_button = driver.find_element(By.XPATH, '//span[@data-icon="send"]')
            send_button.click()
            print(f"Message sent to {name} ({phone})")
            time.sleep(delay)  # Delay before the next message

        except Exception as e:
            print(f"Error sending message to {name}: {e}")

# Step 4: Set up Selenium WebDriver
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    # Use webdriver-manager to automatically handle ChromeDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Main Function
if __name__ == "__main__":
    # Google API and Sheet Details
    credentials_file = r"C:\Users\braag\Desktop\Big Projects\Automate whatsapp\semiotic-garden-423200-h7-19db36845ec6.json"  
    sheet_url = "https://docs.google.com/spreadsheets/d/1KIFDylffgJPkd4yjoqLnYHZvOL8vTKYmsd_S6-y3iTU/edit?gid=0#gid=0"  

    try:
        # Authorize and Fetch Google Sheet
        sheet = authorize_google_sheets(credentials_file, sheet_url)

        # Set up Selenium Driver
        driver = setup_driver()

        # Send WhatsApp Messages
        send_whatsapp_messages(sheet, driver, delay=15)

    except Exception as e:
        print(f"Error: {e}")


Fetched Data from Google Sheet: [{'Name': 'عبود', 'Number': '\u2066+966 54 672 4467\u2069', 'Whatsapp Link': ''}, {'Name': 'جود', 'Number': '\u2066+966 54 262 5601\u2069', 'Whatsapp Link': ''}]
Error sending message to عبود: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@data-icon="send"]"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x004833E3+25059]
	(No symbol) [0x0040CDE4]
	(No symbol) [0x002EBEC3]
	(No symbol) [0x0032FD86]
	(No symbol) [0x0032FFCB]
	(No symbol) [0x0036D952]
	(No symbol) [0x00351F44]
	(No symbol) [0x0036B51E]
	(No symbol) [0x00351C96]
	(No symbol) [0x00323FAC]
	(No symbol) [0x00324F3D]
	GetHandleVerifier [0x00775543+3113795]
	GetHandleVerifier [0x0078A20A+3198986]
	GetHandleVerifier [0x007829E2+3168226]
	GetHandleVerifier [0x00523250+680016]
	(No symbol